# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#I'm fetching the data from my github repo
from azureml.data.dataset_factory import TabularDatasetFactory

cc_data = "https://raw.githubusercontent.com/obinnaonyema/CreditCardChurn_UdacityAZMLCapstone/main/BankChurners.csv"
cc_cust = TabularDatasetFactory.from_delimited_files(path=cc_data, separator=',')
columns_not_needed = ['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2','CLIENTNUM']
cc_final = cc_cust.drop_columns(columns_not_needed)

In [3]:
ws = Workspace.from_config()
experiment_name = 'hyper-cc-churn'

experiment=Experiment(ws, experiment_name)

In [4]:
#set up compute cluster
cluster_name = "compute-obi-3"
try:
    cluster = ComputeTarget(ws, cluster_name)
    print(f"{ccname} exists or is in use!, choose a different name")
except:
    cluster_config = AmlCompute.provisioning_configuration(vm_size="Standard_D4_V2", min_nodes=1, max_nodes=5)
    cluster = ComputeTarget.create(ws, cluster_name, cluster_config)
cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4), "--max_iter": choice(40,80,120,130,200)})

if "outputs" not in os.listdir():
    os.mkdir("./outputs")

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target=cluster, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, policy=early_termination_policy, primary_metric_name="Accuracy",
                                          hyperparameter_sampling=param_sampling,
                                         max_total_runs=50,
                                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [10]:
#TODO: Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(hyperDrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])

Best Run Id:  HD_d1f39567-d6f6-47d9-96aa-9a2afc5961c8_11

 Accuracy: 0.8986508719973676

 Regularization Strength: 4.0

 Max Iterations: 200


In [15]:
best_run.get_file_names()


['azureml-logs/55_azureml-execution-tvmps_04e5af378f1ee89f46b1f5e3f1ce0f8bb9a579b6a77f3cdee0406c23aa63c809_d.txt',
 'azureml-logs/65_job_prep-tvmps_04e5af378f1ee89f46b1f5e3f1ce0f8bb9a579b6a77f3cdee0406c23aa63c809_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_04e5af378f1ee89f46b1f5e3f1ce0f8bb9a579b6a77f3cdee0406c23aa63c809_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/109_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/cchypermodel.pkl']

In [17]:
#TODO: Save the best model
model = best_run.register_model(model_name = 'hyper-cc-churn', model_path = 'outputs/cchypermodel.pkl') 
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-139586', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-139586'), name=hyper-cc-churn, id=hyper-cc-churn:1, version=1, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service